In [24]:
import folium
import networkx as nx
import osmnx as ox
import os
import pandas as pd
from PIL import Image
import time
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx
import geopandas as gpd
from shapely.geometry import LineString, Point
from folium import plugins
from geopy.distance import geodesic
from selenium import webdriver

In [2]:
# Baixe o grafo da região usando OSMnx
place_name = "São Paulo, Brazil"  # Defina o nome da cidade ou região
graph = ox.graph_from_place(place_name, network_type='all')

In [3]:
# Caminho para a pasta com os arquivos
caminho_pasta = r"E:\Dissertacao-v4\Dissertacao\Instancias Clusters"

# Inicializando o dicionário final
lat_long = {}

# Iterando por todos os arquivos na pasta
for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith(".xlsx") and arquivo.startswith("tabelas_"):
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)
        
        # Lendo o arquivo Excel
        df = pd.read_excel(caminho_arquivo)
        
        # Inicializando o dicionário para o arquivo atual
        clusters_dict = {}
        
        # Iterando por clusters únicos
        for cluster in df['Cluster'].unique():
            # Filtrando as linhas do cluster atual
            cluster_data = df[df['Cluster'] == cluster]
            # Extraindo os valores da coluna 'latlng'
            clusters_dict[cluster] = cluster_data['latlng'].tolist()
        
        # Adicionando ao resultado com o nome do arquivo (sem extensão) como chave
        lat_long[arquivo.replace(".xlsx", "")] = clusters_dict

In [4]:
CD = "-23.49549792838335,-46.76044616284469"

# Iterando sobre o dicionário
for arquivo in lat_long:
    for cluster in lat_long[arquivo]:
        # Adicionando o novo elemento no início da lista
        lat_long[arquivo][cluster].insert(0, CD)

In [5]:
# Caminho do arquivo
caminho_arquivo_resultados_Ecs = r"E:\Dissertacao-v4\Dissertacao\RESULTADOS\Ecs\Resultados_Ecs.xlsx"
caminho_arquivo_resultados_Hybrid = r"E:\Dissertacao-v4\Dissertacao\RESULTADOS\Hibrido\Resultados_Hibrido.xlsx"

# Lendo a aba "resumo"
df_resumo_Ecs = pd.read_excel(caminho_arquivo_resultados_Ecs, sheet_name="resumo")
df_resumo_Hybrid = pd.read_excel(caminho_arquivo_resultados_Hybrid, sheet_name="resumo")

# Criando o dicionário final
sequencias_rotas_Ecs = {}
sequencias_rotas_Hybrid = {}

# Extraindo os nomes únicos dos arquivos na coluna "File"
df_resumo_Ecs['File'] = df_resumo_Ecs['File'].str.split('\\').str[-1].str.replace(".xlsx", "")
arquivos_unicos_Ecs = df_resumo_Ecs['File'].unique()

df_resumo_Hybrid['File'] = df_resumo_Hybrid['File'].str.split('\\').str[-1].str.replace(".xlsx", "")
arquivos_unicos_Hybrid = df_resumo_Hybrid['File'].unique()

# MULTIPLE ECS LOOP
for arquivo in arquivos_unicos_Ecs:
    sequencias_rotas_Ecs[arquivo] = {}
    
    # Filtrando as linhas correspondentes ao arquivo atual
    linhas_arquivo_Ecs = df_resumo_Ecs[df_resumo_Ecs['File'] == arquivo]
    
    # Iterando por clusters únicos do arquivo atual
    for cluster in linhas_arquivo_Ecs['cluster'].unique():
        sequencias_rotas_Ecs[arquivo][cluster] = {}
        
        # Filtrando as linhas correspondentes ao cluster atual
        linhas_cluster_Ecs = linhas_arquivo_Ecs[linhas_arquivo_Ecs['cluster'] == cluster]
        
        # Iterando pelas linhas do cluster atual
        for _, linha in linhas_cluster_Ecs.iterrows():
            aba_instancia_Ecs = f"Instancia_{int(linha.name) + 1}"  # Nome da aba correspondente
            
            # Lendo a aba da Instancia
            df_instancia_Ecs = pd.read_excel(caminho_arquivo_resultados_Ecs, sheet_name=aba_instancia_Ecs)
            
            # Filtrando para variáveis "x_ijk"
            df_x_ijk_Ecs = df_instancia_Ecs[df_instancia_Ecs['variavel'].str.contains("x_ijk")]
            
            # Iterando por valores únicos de "k"
            for k in df_x_ijk_Ecs['k'].unique():
                # Filtrando os dados para o valor atual de "k"
                df_k_Ecs = df_x_ijk_Ecs[df_x_ijk_Ecs['k'] == k]
                
                # Criando a sequência de visita
                caminho_Ecs = [0]  # Sempre começa no ponto 0
                for _, row in df_k_Ecs.iterrows():
                    if row['i'] not in caminho_Ecs:
                        caminho_Ecs.append(row['i'])
                    if row['j'] not in caminho_Ecs:
                        caminho_Ecs.append(row['j'])
                caminho_Ecs.append(0)  # Sempre retorna ao ponto 0
                
                # Armazenando no dicionário
                sequencias_rotas_Ecs[arquivo][cluster][k] = caminho_Ecs

# HYBRID LOOP
for arquivo in arquivos_unicos_Hybrid:
    sequencias_rotas_Hybrid[arquivo] = {}
    
    # Filtrando as linhas correspondentes ao arquivo atual
    linhas_arquivo_Hybrid = df_resumo_Hybrid[df_resumo_Hybrid['File'] == arquivo]
    
    # Iterando por clusters únicos do arquivo atual
    for cluster in linhas_arquivo_Hybrid['cluster'].unique():
        sequencias_rotas_Hybrid[arquivo][cluster] = {}
        
        # Filtrando as linhas correspondentes ao cluster atual
        linhas_cluster_Hybrid = linhas_arquivo_Hybrid[linhas_arquivo_Hybrid['cluster'] == cluster]
        
        # Iterando pelas linhas do cluster atual
        for _, linha in linhas_cluster_Hybrid.iterrows():
            aba_instancia_Hybrid = f"Instancia_{int(linha.name) + 1}"  # Nome da aba correspondente
            
            # Lendo a aba da Instancia
            df_instancia_Hybrid = pd.read_excel(caminho_arquivo_resultados_Hybrid, sheet_name=aba_instancia_Hybrid)
            
            # Filtrando para variáveis "x_ijk"
            df_x_ijk_Hybrid = df_instancia_Hybrid[df_instancia_Hybrid['variavel'].str.contains("x_ijk")]
            
            # Iterando por valores únicos de "k"
            for k in df_x_ijk_Hybrid['k'].unique():
                # Filtrando os dados para o valor atual de "k"
                df_k_Hybrid = df_x_ijk_Hybrid[df_x_ijk_Hybrid['k'] == k]
                
                # Criando a sequência de visita
                caminho_Hybrid = [0]  # Sempre começa no ponto 0
                for _, row in df_k_Hybrid.iterrows():
                    if row['i'] not in caminho_Hybrid:
                        caminho_Hybrid.append(row['i'])
                    if row['j'] not in caminho_Hybrid:
                        caminho_Hybrid.append(row['j'])
                caminho_Hybrid.append(0)  # Sempre retorna ao ponto 0
                
                # Armazenando no dicionário
                sequencias_rotas_Hybrid[arquivo][cluster][k] = caminho_Hybrid

In [6]:
#sequencias_rotas_Hybrid['tabelas_2023-11-14'][6][0]

In [7]:
lat_long['tabelas_2023-09-20'][1][0]

'-23.49549792838335,-46.76044616284469'

RODAR A PARTIR DAQUI

In [8]:
dia = "2023-12-08"
cluster = 5
veiculo = 1

In [9]:
def gerar_sequencia(dia, cluster, veiculo, sequencias_rotas, lat_long):
    # Construir as chaves para acessar os dicionários
    key_resultado = f"tabelas_{dia}"
    key_latlong = f"tabelas_{dia}"
    
    # Verificar se as chaves existem nos dicionários
    if key_resultado not in sequencias_rotas or key_latlong not in lat_long:
        raise KeyError(f"Dados para o dia {dia} não encontrados nos dicionários.")
    
    # Verificar se o cluster existe
    if cluster not in sequencias_rotas[key_resultado] or cluster not in lat_long[key_latlong]:
        raise KeyError(f"Cluster {cluster} não encontrado para o dia {dia}.")
    
    # Verificar se o veículo existe no cluster
    if veiculo not in sequencias_rotas[key_resultado][cluster]:
        raise KeyError(f"Veículo {veiculo} não encontrado no cluster {cluster} para o dia {dia}.")
    
    # Obter a sequência de índices do sequencias_rotas
    indices = sequencias_rotas[key_resultado][cluster][veiculo]
    
    # Obter a lista de coordenadas do lat_long e converter para tuplas de floats
    coordenadas_str = lat_long[key_latlong][cluster]
    coordenadas = [
        tuple(map(float, coord.split(','))) for coord in coordenadas_str
    ]
    
    # Gerar a sequência de coordenadas correspondentes aos índices
    sequencia = [coordenadas[i] for i in indices]
    
    return sequencia

# Chamada da função
sequencia_Ecs = gerar_sequencia(dia, cluster, veiculo, sequencias_rotas_Ecs, lat_long)
sequencia_Hybrid = gerar_sequencia(dia, cluster, veiculo, sequencias_rotas_Hybrid, lat_long)

print(sequencia_Ecs)
print()
print(sequencia_Hybrid)

[(-23.49549792838335, -46.76044616284469), (-23.673671687, -46.678195844), (-23.6098467, -46.666178387), (-23.60947, -46.66641), (-23.6096646, -46.666957), (-23.6061305, -46.672201), (-23.6096351, -46.66684974), (-23.60993, -46.66687), (-23.60932733, -46.66690341), (-23.61177, -46.66852), (-23.62596, -46.67207), (-23.627486, -46.674804), (-23.626265729, -46.671808732), (-23.627739890434, -46.6712049318335), (-23.6258616446958, -46.6684038024771), (-23.62944, -46.67055), (-23.623947, -46.678237), (-23.62282, -46.67256), (-23.620923464, -46.67135583), (-23.63268, -46.65589), (-23.604117, -46.672142), (-23.49549792838335, -46.76044616284469)]

[(-23.49549792838335, -46.76044616284469), (-23.6016154710831, -46.6642949799505), (-23.6098467, -46.666178387), (-23.60947, -46.66641), (-23.6061305, -46.672201), (-23.61202234, -46.659811), (-23.6128583, -46.662071165), (-23.608404646, -46.659283426), (-23.601703847, -46.661833715), (-23.60298249, -46.66351228), (-23.60739197, -46.6598516), (-23.5

In [10]:
# Montar a lista de latitudes e longitudes conforme a sequência
latlng_sequence_Ecs = sequencia_Ecs
latlng_sequence_Hybrid = sequencia_Hybrid

In [ ]:
rota = latlng_sequence_Ecs

# Função para calcular os nós mais próximos e plotar as rotas
def plot_route(route, color):
    total_distance = 0  # Inicializar a distância total

    # Calcule os nós mais próximos para cada ponto da rota
    nodes = [ox.distance.nearest_nodes(graph, lon, lat) for lat, lon in route]
    
    # Calcule a rota e adicione ao mapa
    for i in range(len(nodes) - 1):
        # Calcule a rota entre dois nós consecutivos
        path = nx.shortest_path(graph, nodes[i], nodes[i + 1], weight='length')
        
        # Calcular a distância entre os nós consecutivos
        distance = nx.shortest_path_length(graph, nodes[i], nodes[i + 1], weight='length')
        total_distance += distance  # Acumular a distância total
        
        # Adicione a rota ao mapa com a cor especificada
        folium.PolyLine([(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in path],
                        color=color, weight=5, opacity=0.7).add_to(m)
    
    # Adicionar marcadores para cada ponto
    for lat, lon in route:
        folium.CircleMarker(
            location=[lat, lon], 
            radius=6,        # Tamanho da bolinha
            color="black",   # Cor do contorno
            fill=True,       # Preencher o círculo
            fill_color=color, # Cor de preenchimento igual à cor da rota
            fill_opacity=0.7 # Opacidade do preenchimento
        ).add_to(m)
    
    # Imprimir a distância total percorrida
    print(f"Distância total percorrida: {total_distance:.2f} metros")

# Gerar o mapa com Folium (centrado em torno da média das rotas)
center_lat = rota[0][0]
center_lon = rota[0][1]
m = folium.Map(
    location=[center_lat, center_lon], 
    zoom_start=14, 
    tiles='cartodb positron',  # Use "cartodb positron" para 2D
    control_scale=True         # Ativar o controle de escala no mapa
)

# Plotar a primeira rota (em azul)
plot_route(rota, color="blue")

file_name = f"rota_{dia}_cluster{cluster}_veic{veiculo}_Ecs.html"

# Salvar o mapa
m.save(file_name)

# Confirmação
print(f"Mapa salvo como {file_name}")

In [ ]:
rota = latlng_sequence_Hybrid

# Função para calcular os nós mais próximos e plotar as rotas
def plot_route(route, color):
    total_distance = 0  # Inicializar a distância total

    # Calcule os nós mais próximos para cada ponto da rota
    nodes = [ox.distance.nearest_nodes(graph, lon, lat) for lat, lon in route]
    
    # Calcule a rota e adicione ao mapa
    for i in range(len(nodes) - 1):
        # Calcule a rota entre dois nós consecutivos
        path = nx.shortest_path(graph, nodes[i], nodes[i + 1], weight='length')
        
        # Calcular a distância entre os nós consecutivos
        distance = nx.shortest_path_length(graph, nodes[i], nodes[i + 1], weight='length')
        total_distance += distance  # Acumular a distância total
        
        # Adicione a rota ao mapa com a cor especificada
        folium.PolyLine([(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in path],
                        color=color, weight=5, opacity=0.7).add_to(m)
    
    # Adicionar marcadores para cada ponto
    for lat, lon in route:
        folium.CircleMarker(
            location=[lat, lon], 
            radius=6,        # Tamanho da bolinha
            color="black",   # Cor do contorno
            fill=True,       # Preencher o círculo
            fill_color=color, # Cor de preenchimento igual à cor da rota
            fill_opacity=0.7 # Opacidade do preenchimento
        ).add_to(m)
    
    # Imprimir a distância total percorrida
    print(f"Distância total percorrida: {total_distance:.2f} metros")

# Gerar o mapa com Folium (centrado em torno da média das rotas)
center_lat = rota[0][0]
center_lon = rota[0][1]
m = folium.Map(
    location=[center_lat, center_lon], 
    zoom_start=14, 
    tiles='cartodb positron',  # Use "cartodb positron" para 2D
    control_scale=True         # Ativar o controle de escala no mapa
)

# Plotar a primeira rota (em azul)
plot_route(rota, color="red")

file_name = f"rota_{dia}_cluster{cluster}_veic{veiculo}_Hybrid.html"

# Salvar o mapa
m.save(file_name)

# Confirmação
print(f"Mapa salvo como {file_name}")